# Opean AI Gymnasium 라이브러리 예제 코드

이 예제는 gymnasium 문서를 참고하여 작성되었습니다.
Documentation: https://gymnasium.farama.org/

## 1. Open AI Gymnasium 라이브러리 소개
- Gymnasium은 single agent reinforcement learning 환경을 위한 API를 제공하는 라이브러리 입니다.
- cartpole, pendulum, ... mujoco(로봇 물리엔진), atari(게임) 등 기본 환경도 제공하며, 사용자가 직접 환경을 만들어서 사용할 수도 있습니다.

### 라이브러리 설치
``` bash
pip install gymnasium
```

In [1]:
# 라이브러리 import
import gymnasium as gym

ModuleNotFoundError: No module named 'gymnasium'


## 2. 핵심 API

환경은 `Env` 클래스와 `Wrappers`로 구현이 됩니다.
- `Env` : Markov Decision Process(MDP)를 표현하는 Python Class
- `Wrappers` : 환경을 감싸서 유용한 기능들을 추가해주는 Python Wrapper Class


## 3. 환경 객체 생성

- `gym.envs.registry.keys()` : `gym.make()`로 생성할 수 있는 환경 목록을 확인합니다.

In [2]:
envs = list(gym.envs.registry.keys())
print(len(envs))

for i in range(len(envs)):
    print(envs[i], end=' || ')
    if i % 5 == 4:
        print()

51
CartPole-v0 || CartPole-v1 || MountainCar-v0 || MountainCarContinuous-v0 || Pendulum-v1 || 
Acrobot-v1 || phys2d/CartPole-v0 || phys2d/CartPole-v1 || phys2d/Pendulum-v0 || LunarLander-v2 || 
LunarLanderContinuous-v2 || BipedalWalker-v3 || BipedalWalkerHardcore-v3 || CarRacing-v2 || Blackjack-v1 || 
FrozenLake-v1 || FrozenLake8x8-v1 || CliffWalking-v0 || Taxi-v3 || tabular/Blackjack-v0 || 
tabular/CliffWalking-v0 || Reacher-v2 || Reacher-v4 || Pusher-v2 || Pusher-v4 || 
InvertedPendulum-v2 || InvertedPendulum-v4 || InvertedDoublePendulum-v2 || InvertedDoublePendulum-v4 || HalfCheetah-v2 || 
HalfCheetah-v3 || HalfCheetah-v4 || Hopper-v2 || Hopper-v3 || Hopper-v4 || 
Swimmer-v2 || Swimmer-v3 || Swimmer-v4 || Walker2d-v2 || Walker2d-v3 || 
Walker2d-v4 || Ant-v2 || Ant-v3 || Ant-v4 || Humanoid-v2 || 
Humanoid-v3 || Humanoid-v4 || HumanoidStandup-v2 || HumanoidStandup-v4 || GymV21Environment-v0 || 
GymV26Environment-v0 || 

- gym.make('환경이름') : 환경을 생성합니다.

user가 사용할 수 있는 `Env` 객체를 리턴합니다.

In [3]:
env_name = "LunarLander-v2"
env = gym.make(env_name, render_mode="rgb_array")

## 4. 환경과 상호작용

In [4]:
# pygame window와 Jupyter Notebook의 호환성이 좋지 않아
# 따로 moviepy 라이브러리를 사용하여 비디오를 저장하는 코드입니다.
import moviepy.editor as mpy
import os
# List to store frames
frames = []

In [5]:
observation, info = env.reset()

for _ in range(1000):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)
    frame = env.render()

    # moviepy 코드
    frames.append(frame)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

# moviepy 코드
# Ensure the directory exists
video_dir = './videos'
os.makedirs(video_dir, exist_ok=True)

# Create a video from the frames
clip = mpy.ImageSequenceClip(frames, fps=30)
clip.write_videofile(f"{video_dir}/{env_name}.mp4")

Moviepy - Building video ./videos/LunarLander-v2.mp4.
Moviepy - Writing video ./videos/LunarLander-v2.mp4



Moviepy - Done !
Moviepy - video ready ./videos/LunarLander-v2.mp4


In [6]:
from IPython.display import Video

# Path to your video file
video_path = f"{video_dir}/{env_name}.mp4"

# Display the video
Video(video_path, embed=True)

### `env.reset()`

- 환경을 초기화 하여 초기 상태로 되돌아갑니다.
- 초기 observation과 info를 리턴합니다.

파라미터 :
- `seed` (optional int) : 환경의 PRNG(np_random)을 초기화 하기 위한 seed 값.
    - 만약 환경이 PRNG를 가지고 있지 않고, `seed=None`(default)가 전달되면, seed가 자동으로 선택됩니다. (e.g. timestamp, /dev/urandom)
    - 만약 환경이 PRNG를 가지고 있고, `seed=None`(default)가 전달되면, PRNG가 초기화 되지 않습니다.
    - 만약 `seed`에 정수값이 전달되면, PRNG가 이미 존재하더라도 해당 값으로 초기화 됩니다.
    - 따라서, 일반적으로 환경 객체를 생성한 직후에만 seed를 전달하고, 그 이후에는 seed를 전달하지 않습니다.
- `options` (optional dict) : 환경을 초기화 하기 위한 추가적인 옵션들을 전달합니다. (특정 환경에서만 사용됩니다.)

리턴값 :
- `observation`(ObsType) : 초기 상태의 observation, $O_0$.
    - 이는 `observation_space`의 원소여야합니다.
    - `step()`에 의해 리턴되는 observation과 같은 형태여야합니다.
- `info`(dict) : 초기 상태의 info.
    - `observation`을 보완하는 추가적인 정보가 담겨있습니다.
    - `observation`과는 달리 agent에게 직접적인 영향을 주지 않습니다.
    - 이는 `step()`에 의해 리턴되는 info와 같은 형태여야합니다.


### `env.step(action)`

- agent의 action을 이용하여 환경을 한 step 진행합니다.
- episode가 끝나면(terminated or truncated), 다음 episode를 진행하기 위해서 다시 `env.reset()`을 호출하여 환경의 상태를 초기화합니다.

리턴값 : 
- `observation`(ObsType) : agent의 action에 따른 다음 상태의 observation, $O_{t+1}$.
    - 이는 `observation_space`의 원소여야합니다.
- `reward`(SupportsFloat) : agent의 action에 따른 reward, $R_{t+1}$.
- `terminated`(bool) : agent가 terminal state(MDP에 정의된)에 도달했는지 여부.
    - 만약 `terminated=True`이면, `reset()`을 호출하여 다음 episode를 진행합니다.
- `truncated`(bool) : truncation 조건(MDP에 정의 되지 않은)을 만조했는지 여부.
    - 일반적으로, 시간초과(timestep 초과)로 인한 episode 종료를 의미합니다.
    - 만약 `truncated=True`이면, `reset()`을 호출하여 다음 episode를 진행합니다.
- `info`(dict) : 추가적인 정보를 담은 dictionary.
    - 디버깅, 학습, 로깅에 도움이 될 수 있는 정보를 담습니다.
    - ex) agent의 성능 지표, observation에 포함되지 않은 정보, 최종 reward를 구성하는 개별 reward 등.

### `env.render()`

- `render_mode`에 따라 환경을 시각화합니다.
- `render_mode`는 환경에 따라 다르며, `env.metadata['render_modes']`를 통해 확인할 수 있습니다.


In [7]:
env.metadata['render_modes']

['human', 'rgb_array']



- `env.close()` : 환경을 종료합니다.

## 5. Action space, Observation space

모든 환경은 `Space`클래스를 상속하는 클래스의 객체인 `action_space`와 `observation_space`를 가져야합니다.

`env.action_space`

- 환경에서 valid한 action의 형식을 정의합니다.

`env.observation_space`

- 환경에서 observation의 형식을 정의합니다.

In [8]:
env.action_space

Discrete(4)

In [9]:
env.observation_space

Box([-1.5       -1.5       -5.        -5.        -3.1415927 -5.
 -0.        -0.       ], [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ], (8,), float32)


### Fundamental Spaces

#### `Box`

- A(possibly unbounded) box in $\mathcal R^n$.
- Box는 n개의 closed interval의 Cartesian product로 정의됩니다. 
    - 각 interval은 다음 중 하나의 형태입니다. 
        - $[a, b]$, $(-\infty, b]$, $[a, \infty)$, $(-\infty, \infty)$

파라미터 :
- `low`(SupportsFloat | np.ndarray) : 각 차원의 lower bound.
- `high`(SupportsFloat | np.ndarray) : 각 차원의 upper bound.
- `shape`(Optional) : Box의 shape.
- `dtype` : Box의 data type. (default : `np.float32`)
- `seed`(Optional) : Box의 PRNG를 초기화 하기 위한 seed 값.


In [10]:
import numpy as np
from gymnasium.spaces import Box

In [11]:
# 모든 차원에서 같은 bound
Box(low=-1.0, high=2.0, shape=(3, 4), dtype=np.float32)

Box(-1.0, 2.0, (3, 4), float32)

In [12]:
# 차원마다 다른 bound
Box(low=np.array([-1.0, -2.0]), high=np.array([2.0, 4.0]), dtype=np.float32)

Box([-1. -2.], [2. 4.], (2,), float32)

#### `Discrete`

- 유한한 정수 집합을 나타냅니다.
    - $\{a, a + 1, ..., a + n - 1\}$

파라미터 :
- `n`(int) : 정수 개수
- `seed`(Optional) : PRNG를 초기화 하기 위한 seed 값.
- `start`(int) : 첫번째 정수의 값. (default : 0)


In [13]:
from gymnasium.spaces import Discrete

In [14]:
discrete = Discrete(2, seed=42) # {0, 1}
discrete.sample()

np.int64(0)

In [15]:
discrete = Discrete(3, start=-1, seed=42) # {-1, 0, 1}
discrete.sample()

np.int64(-1)

#### `MultiDiscrete`

- `Discrete` space들의 cartesian product.

e.g. 닌텐도 게임 컨트롤러 - 3개의 discrete action space로 구성
1. 방향키 : Discrete(5) - NOOP[0], UP[1], RIGHT[2], DOWN[3], LEFT[4] - min=0, max=4
2. A 버튼 : Discrete(2) - NOOP[0], Pressed[1] - min=0, max=1
3. B 버튼 : Discrete(2) - NOOP[0], Pressed[1] - min=0, max=1

위 구성은 `MultiDiscrete([5, 2, 2])`로 표현할 수 있습니다.

*사실 `Discrete(20)`으로 표현할 수도 있음.


파라미터 :
- `nvec` : 각 categorical 변수의 개수를 담은 vector. 각 위치의 값은 해당 변수의 정수 개수를 나타냅니다.
    - 일반적으로 정수 리스트
    - 하지만, space가 여러 개의 축을 가질때, 더 복잡한 numpy array가 필요할 수 있습니다.
- `dtype` : 정수 타입 중 하나
- `seed`(Optional) : PRNG를 초기화 하기 위한 seed 값.
- `start`(Optional) : 각 categorical 변수의 첫번째 정수의 값. (default : 0)

In [16]:
from gymnasium.spaces import MultiDiscrete

In [17]:
multi_discrete = MultiDiscrete([[1, 2], [3, 4]])
multi_discrete.sample()

array([[0, 0],
       [0, 1]])

#### `MultiBinary`

- n-shape binary space. (n개의 binary 값)

파라미터 :
- `n` : space의 shape을 결정합니다.
    - 정수 값 : flatten space
    - tuple, list, 혹은 np.ndarray : 다차원 space
- `seed`(Optional) : PRNG를 초기화 하기 위한 seed 값.


In [18]:
from gymnasium.spaces import MultiBinary

In [19]:
multi_binary = MultiBinary(5, seed=42)
multi_binary.sample()

array([1, 0, 1, 0, 1], dtype=int8)

In [20]:
multi_binary = MultiBinary([3, 2], seed=42)
multi_binary.sample()

array([[1, 0],
       [1, 0],
       [1, 1]], dtype=int8)

이 외에도 string을 다루는 `Text`가 있지만, 사용 빈도가 낮습니다.

### Composite Spaces

#### `Dict`

- 여러 space들을 dictionary로 묶은 space.

파라미터 :
- `spaces` : dictionary의 key-value 쌍으로, 각 value는 `Space` 객체여야합니다.
- `seed`(Optional) : PRNG를 초기화 하기 위한 seed 값.
- `**spaces_kwargs` : 각 space에 대한 추가적인 옵션들을 전달합니다.


In [21]:
from gymnasium.spaces import Dict

In [22]:
dict_space = Dict({"position": Box(-1, 1, shape=(2,)), "color": Discrete(3)}, seed=42)
dict_space.sample()

OrderedDict([('color', np.int64(0)),
             ('position', array([-0.3991573 ,  0.21649833], dtype=float32))])

이 외에도 `Tuple`, `Sequence`, `Graph` 가 있지만, 사용 빈도가 낮습니다.

## 6. 과제

- 에제에 사용된 `LunarLander-v2` 말고도 다양한 환경이 구현되어있습니다.
- 아래의 링크에서 다양한 환경의 Action Space, Observation Space 등을 확인하고, 환경을 생성하여 random agent를 구현해보세요.

https://gymnasium.farama.org/environments/classic_control/